# Find an Maximum tolerated dose (MTD) for a phrase I trial

* 20 patients are enrolled, the probability that a patient would experience DLT can be simulated via a uniform distribution on [0,1].
* Target TLL=0.25
* $DTL = 1$ if $u \leq True[MTD]$, else $0$ <br>  where $ u \sim uniform[0,1]$

In [2]:
install.packages('CRM')

package 'CRM' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\shims\AppData\Local\Temp\Rtmp2tEhiV\downloaded_packages


In [3]:
library(CRM)

Continual Reassessment Method (CRM) for Phase I Clinical Trials

https://cran.r-project.org/web/packages/CRM/CRM.pdf

## 1. Data preperation

* prior : prior probability
* d: dose level , we use hyperbolic tangent model with a=1.
* true: True probability

In [5]:
prior <- c(0.04, 0.11, 0.25, 0.42, 0.58, 0.7, 0.8)
d <- -log(1/(prior)-1)/2 # hyperbolic tangent model, a=1.
true <- c(0.02, 0.07, 0.2, 0.39, 0.57, 0.7, 0.78)

## 2. Run simulation

In [11]:
ptdata <- c()
u <- c()
MTD_true<- c()

set.seed(123)

for(i in 1:20) {
      if (i==1) {
        u[i] <- runif(1,0,1)
        MTD_true[i]<-true[1]
        DLT <- sum(u<=true[1])
        ptdata <- rbind(ptdata, c(1,DLT))
        res <- CRM::crm(target=0.25, prior, ptdata, model=1)
        a <- res$a
        prob.hyperbolic <-(exp(d)/(exp(d)+exp(-d)))^a
        res$MTD
  } else {
        u[i] <- runif(1,0,1)
        DLT <- sum(u<=true[res$MTD])
        MTD_true[i]<-true[res$MTD]
        ptdata <- rbind(ptdata, c(res$MTD,DLT))
        res <- CRM::crm(target=0.25, prior, ptdata, model=1)
        a <- res$a
        prob.hyperbolic <-(exp(d)/(exp(d)+exp(-d)))^a
        res$MTD
  }  
}

In [12]:
patient_data<- cbind(u,MTD_true,ptdata)
colnames(patient_data) <- c("u","true[MTD]","MTD","DLT")

patient_data


u,true[MTD],MTD,DLT
0.28757752,0.02,1,0
0.78830514,0.20,3,0
0.40897692,0.39,4,1
0.88301740,0.20,3,0
0.94046728,0.20,3,0
0.04555650,0.20,3,1
0.52810549,0.20,3,1
0.89241904,0.07,2,1
0.55143501,0.02,1,0
0.45661474,0.07,2,1


Result: MTD=1 by the 20 patient's simlation.